In [2]:
import json
import pandas as pd
import traceback

In [3]:
df=pd.read_excel('data.xlsx')
c0=df.columns
df.head(1)

,序号,标注人员,标注结果
0,1,王小二,"[{""tags"":[""123"",""231""],""cc"":{""tags"":54,""data"":..."


In [4]:
def df_to_json(df):
    #找到json列
    jc=''
    for c in df.columns:
        a0=str(list(df[c]))
        if '{' in a0 and ':' in a0 and '[' in a0:
            jc=c
    if jc=='':
        return []
    res=[]
    for i,c in df.iterrows():
        r0={}
        for col in df.columns:
            if col!=jc:
                r0[col]=c[col]
            else:
                try:
                    if type(c[col])!=str:
                        r0[col]=[]
                    else:
                        r0[col]=json.loads(c[col])
                except:
                    print(traceback.format_exc())
        res.append(r0.copy())
    return res
r00=df_to_json(df)

In [35]:
def unfolds(js,k0='',kvs=[]):
    ojs={}
    for k,v in js.items():
        if type(v)==dict:
            ojs1=ojs.copy()
            ojs.update(unfolds(v,k,ojs1.keys()))
        else:
            if k in kvs:
                ojs['{0}_{1}'.format(k0,k)]=v
            elif ojs.get(k,0)==0:
                ojs[k]=v
            else:#有这一个k了
                if k0=='':
                    ojs[k]=v
                else:
                    ojs['{0}_{1}'.format(k0,k)]=v
    return ojs

In [41]:
def unfolds(js,k0=''):
    ojs={}
    for k,v in js.items():
        if type(v)==dict:
            ojs.update(unfolds(v,k))
        elif k0=='':
            ojs[k]=v
        else:
            ojs['{0}_{1}'.format(k0,k)]=v
    return ojs

In [42]:
def ex_aa(aa,bb,cc=''):
    f=0
    rc=bb.copy()
    aa=unfolds(aa,cc)
    ks=[]
    for k,v in aa.items():
        if type(v)!=list:
            if rc.get(k,0)==0:
                rc[k]=v
            else:
                rc['{0}_{1}'.format(cc,k)]=v
        elif v==[]:
            pass
            #rc[k]='[]'
        else:#list
            if type(v[0])==dict:
                f+=1
                ks.append(k) #假设有多个列表的情况
            else:
                if rc.get(k,0)==0:
                    rc[k]=str(v)
                else:
                    rc['{0}_{1}'.format(cc,k)]=str(v)
    if f==0:
        rc.update(bb)
        return [rc]
    else:
        w=[]
        for i in aa[ks[0]]:
            w.extend(ex_aa(i,rc,ks[0]))
        return w

In [43]:
res=[]
for i in r00:
    res.extend(ex_aa(i,{}))
res

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['123', '231']",
  'cc_tags': 54,
  'cc_data': 'cc',
  '标注结果_username': '张三',
  '标注结果_infos_地点': '北京',
  '标注结果_infos_爱好': '足球'},
 {'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['123', '231']",
  'cc_tags': 54,
  'cc_data': 'cc',
  '标注结果_username': '张三',
  '标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '篮球'},
 {'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['23', '2931']",
  '标注结果_username': '火星',
  '标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '足球'},
 {'序号': 2, '标注人员': '王小二'},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果_tags': "['96', 'f2931']",
  '标注结果_username': '楼满月',
  '标注结果_infos_地点': '湖北',
  '标注结果_infos_爱好': '足球'}]

In [25]:
r00

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果': [{'tags': ['123', '231'],
    'cc': {'tags': 54, 'data': 'cc'},
    'username': '张三',
    'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
   {'tags': ['23', '2931'],
    'username': '火星',
    'infos': [{'地点': '天津', '爱好': '足球'}]}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': []},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果': {'tags': ['96', 'f2931'],
   'username': '楼满月',
   'infos': [{'地点': '湖北', '爱好': '足球'}]}}]

In [44]:
pd.DataFrame(res)

,cc_data,cc_tags,序号,标注人员,标注结果_infos_地点,标注结果_infos_爱好,标注结果_tags,标注结果_username
0,cc,54.0,1,王小二,北京,足球,"['123', '231']",张三
1,cc,54.0,1,王小二,天津,篮球,"['123', '231']",张三
2,NaN,NaN,1,王小二,天津,足球,"['23', '2931']",火星
3,NaN,NaN,2,王小二,NaN,NaN,NaN,NaN
4,NaN,NaN,3,杨六,湖北,足球,"['96', 'f2931']",楼满月


In [ ]:
out=pd.DataFrame(res)

In [ ]:
for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]

In [ ]:
out.to_excel('data-out.xlsx',index=False)

In [20]:
r00

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果': [{'tags': ['123', '231'],
    'cc': {'tags': 54, 'data': 'cc'},
    'username': '张三',
    'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
   {'tags': ['23', '2931'],
    'username': '火星',
    'infos': [{'地点': '天津', '爱好': '足球'}]}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': []},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果': {'tags': ['96', 'f2931'],
   'username': '楼满月',
   'infos': [{'地点': '湖北', '爱好': '足球'}]}}]

In [41]:
aa={'a':[{'aa':1,'bb':[{'aaa':31,'bbb':32},{'aaa':41,'bbb':42}]},{'aa':11}],'b':2}
ex_aa(aa,{})

[{'b': 2, 'aa': 1, 'aaa': 31, 'bbb': 32},
 {'b': 2, 'aa': 1, 'aaa': 41, 'bbb': 42},
 {'b': 2, 'aa': 11}]

In [34]:
aa={'a':[{'aa':1,'bb':[{'aaa':31,'bbb':32},{'aaa':41,'bbb':42}]},{'aa':11}],'b':2}
'a' in aa.keys()

True